<a href="https://colab.research.google.com/github/BaylorWhitehead/canLstmTransferLearning/blob/main/TranferLeaningAutomationLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports and Drive Mounting**

In [ ]:
# Imports for training
from keras.models import Sequential
from keras.layers import SimpleRNN, Flatten, TimeDistributed, LSTM, Dense
from keras import applications, Sequential, utils
from keras.layers import Dense, BatchNormalization, Dropout, TimeDistributed, LSTM, Flatten, Conv2D, MaxPooling2D, ConvLSTM2D, Conv3D, Activation
from keras.regularizers import L1L2
from tensorflow.keras import regularizers
from keras import losses
import tensorflow as tf
# Transfer learn
from keras.layers import GlobalAveragePooling2D, Input
from keras import Model
from keras.models import load_model
from keras.callbacks import LambdaCallback
from sklearn.metrics import classification_report

# Various
import random
import datetime
import os
import json
import time

# Imports for visualizing training history
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy

# Pandas and Numpy
import numpy as np
import pandas as pd

# Load the Drive helper and mount
from google.colab import drive

# Mount Drive
drive.mount('/content/gdrive')

random.seed(5)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Data Objects

In [ ]:
class SpecificTrainingInfo:
  def __init__(self, epochs, train_test_split, train_val_split, learning_rate):
    self.epochs = epochs
    self.train_test_split = train_test_split
    self.train_val_split = train_val_split
    self.learning_rate = learning_rate

class TrainingInfo:
  def __init__(self, index, steps, features, seq, substeps, batch_size, base_transfer_split, total_data_size, attack_type, base_training_info, transfer_training_info):
    self.index = index
    self.steps = steps
    self.features = features
    self.seq = seq
    self.substeps = substeps
    self.batch_size= batch_size
    self.base_transfer_split = base_transfer_split
    self.num_classes = 2
    self.total_data_size = total_data_size
    self.attack_type = attack_type
    self.base_training_info = base_training_info
    self.transfer_training_info = transfer_training_info

  def __str__(self):
    return f"Attack_{self.attack_type}-steps_{self.steps}-feat_{self.features}-seq_{self.seq}-substeps_{self.substeps}-batchsize_{self.batch_size}-classes_2-basetransplit_{self.base_transfer_split}-datasize_{self.total_data_size}"\
            +f"-baseepoch_{self.base_training_info.epochs}-basettsplit_{self.base_training_info.train_test_split}-basetvsplit_{self.base_training_info.train_val_split}"\
            +f"-transepoch_{self.transfer_training_info.epochs}-transttsplit_{self.transfer_training_info.train_test_split}-transtvsplit_{self.transfer_training_info.train_val_split}"

def generate_training_info_list_from_csv(csv_file_path):
  training_info_list = []
  df = pd.read_csv(csv_file_path)
  for index, row in df.iterrows():
    base_training_info = SpecificTrainingInfo(row['base_epochs'], row['base_train_test_split'], row['base_train_val_split'], row['base_learning_rate'])
    transfer_training_info = SpecificTrainingInfo(row['transfer_epochs'], row['transfer_train_test_split'], row['transfer_train_val_split'], row['transfer_learning_rate'])
    training_info = TrainingInfo(index, row['steps'], row['features'], row['seq'], row['substeps'], row['batch_size'], row['base_transfer_split'], row['total_data_size'], row['attack_type'], base_training_info, transfer_training_info)
    training_info_list.append(training_info)
  return training_info_list

def generate_folder_name_from_training_info(training_info):
  now = datetime.datetime.now()
  short_datetime_string = now.strftime("_%y%m%d-%H%M%S")
  return str(training_info) +short_datetime_string

def save_training_info_as_json(training_info, folder_path, filename):
  """Saves the training info object as a JSON file.

  Args:
    training_info: The TrainingInfo object to save.
    folder_path: The path to the folder where the JSON file should be saved.
    filename: The name of the JSON file.
  """

  def obj_to_dict(obj):
    # Convert object to dictionary, handling nested objects
    if isinstance(obj, TrainingInfo) or isinstance(obj, SpecificTrainingInfo):
      return {k: obj_to_dict(v) for k, v in vars(obj).items()}
    else:
      return obj

  # Convert TrainingInfo object to dictionary
  training_info_dict = obj_to_dict(training_info)

  # Save dictionary to JSON file
  filepath = os.path.join(folder_path, filename)
  with open(filepath, 'w') as f:
    json.dump(training_info_dict, f, indent=4)


# Data Collection and Formatting

In [ ]:
def split_data(df, split):

    base_size = int(split * len(df))

    first_df = df.iloc[0:base_size]
    second_df = df.iloc[base_size:]

    return first_df, second_df

def split_np_data(np_object, split):
    train_size = int(split * len(np_object))

    train_ds = np_object[:train_size]
    val_ds = np_object[train_size:]

    return train_ds, val_ds

def add_trim_data(df_normal, df_attack):

  df_base_tran = df_normal._append(df_attack)

  # Add trim messages
  trim_messages_1 = pd.DataFrame(0, index=np.arange(int(len(df_base_tran)//20)), columns=['time', 'id', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8','label'])

  # Trim sample set
  val = 0
  for i in range (int(len(df_base_tran)//20)): # 1/20 of the messages in base_trim_data set
    trim_messages_1.at[i, 'time'] = 200 + (int(random.random()*100) - 50)
    trim_messages_1.at[i, 'id'] = 432

    val += 1
    if val > 10000:
      val = 0

    temp1_val = val + (int(random.random()*10) - 5)
    trim_messages_1.at[i, 'x1'] = temp1_val & 0x00FF
    trim_messages_1.at[i, 'x2'] = (temp1_val & 0xFF00) >> 8

    temp2_val = val + (int(random.random()*6) - 3)
    trim_messages_1.at[i, 'x3'] = temp2_val & 0x00FF
    trim_messages_1.at[i, 'x4'] = (temp2_val & 0xFF00) >> 8

    trim_messages_1.at[i, 'label'] = 0

  # Combine into df_normal_trim every 20th spot
  combined_normal_up_down_trim = pd.DataFrame(0, index=np.arange(len(df_base_tran)+len(trim_messages_1)), columns=['time', 'id', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8','label'])

  normal_index = 0
  trim_index = 0
  for i in range(len(combined_normal_up_down_trim)-100):
    if i%21 == 0:
      trim_index += 1
      combined_normal_up_down_trim.iloc[i] = trim_messages_1.iloc[trim_index]
    else:
      normal_index += 1
      combined_normal_up_down_trim.iloc[i] = df_base_tran.iloc[normal_index]

  return combined_normal_up_down_trim

def reshape_data(df, training_info, train_test_split):
  df_x, df_y = df.iloc[:, :-1], df.iloc[:, [-1]]

  # Calculate rows
  samples = int(len(df_x)//(training_info.seq))
  head_value =  samples*training_info.seq

  # Reshape Base Data for LSTM
  np_reshape_x = df_x.head(head_value).to_numpy().reshape(samples, training_info.seq, training_info.features)
  # np_reshape_y = df_y.head(head_value).to_numpy().reshape(rows, training_info.seq, 1)
  # np_reshape_y = np.repeat(np_reshape_y, training_info.num_classes, axis=2)
  np_reshape_y = df_y.head(head_value).to_numpy().reshape(samples, training_info.seq, 1).max(axis=1)

  # Shuffle the groups of messages
  shuffler = np.random.permutation(len(np_reshape_x))
  np_reshape_x = np_reshape_x[shuffler]
  np_reshape_y = np_reshape_y[shuffler]
  np_reshape_y = np_reshape_y.flatten()

  # Catagorize
  #np_reshape_y_catagorical = tf.keras.utils.to_categorical(np_reshape_y, num_classes=2)
  display(np_reshape_y)

  # Split data for train and test
  np_train_x, np_test_x = split_np_data(np_reshape_x, train_test_split)
  np_train_y, np_test_y = split_np_data(np_reshape_y, train_test_split)

  return (np_train_x, np_test_x, np_train_y, np_test_y)

def get_formated_and_split_data(training_info):
  # Grab attack free data and set data types
  df_normal = pd.read_csv('gdrive/My Drive/Thesis/CAN Data/normal_run_data.txt',
                sep="\s+", # Separator is whitespace
                header=None, # No header
                usecols=[1,3,6,7,8,9,10,11,12,13,14], #parse only certian columns
                names=['time', 'id', 'dlc', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8']) # Set columns names
  df_normal['time'] = df_normal['time'].apply(lambda x: x*1000000) # Make time not have a decimal point
  df_normal = df_normal.fillna(0).assign(label='0').astype({"time":int, "id":str, "x1":str, "x2":str, "x3":str, "x4":str, "x5":str, "x6":str, "x7":str, "x8":str, "label":int}).apply(lambda x: x.apply(int, base=16) if x.name in ['id', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8'] else x)
  df_normal = df_normal.drop(['dlc'], axis=1)
  df_normal['time'] = df_normal['time'].diff() # Time as difference since last message
  df_normal = df_normal.iloc[1: , :].astype({"time":int})
  df_normal = df_normal[:int(training_info.total_data_size*.6)]

  # Pick attack data
  file_path = 'gdrive/My Drive/Thesis/CAN Data/'
  if training_info.attack_type == "fuzz":
    file_path += 'Fuzzy_dataset_filled.csv'
  elif training_info.attack_type == "dos":
    file_path += 'DoS_dataset_filled.csv'
  elif training_info.attack_type == "gear":
    file_path += 'gear_dataset_filled.csv'
  elif training_info.attack_type == "rpm":
    file_path += 'RPM_dataset_filled.csv'
  else:
    raise Exception("Invalid attack type")

  # Grab attack data and set data types
  df_attack = pd.read_csv(file_path, names=['time', 'id', 'dlc', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8','label']).astype({"dlc": int})
  df_attack['time'] = df_attack['time'].apply(lambda x: x*1000000)
  df_attack['label'] = df_attack['label'].apply(lambda x: 0 if (x == 'R') else 1)
  df_attack = df_attack.astype({"dlc": int, "time":int, "id":str, "x1":str, "x2":str, "x3":str, "x4":str, "x5":str, "x6":str, "x7":str, "x8":str, "label":int}).apply(lambda x: x.apply(int, base=16) if x.name in ["id", 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8'] else x)
  df_attack = df_attack.drop(['dlc'], axis=1)
  df_attack['time'] = df_attack['time'].diff() # Time as difference since last message
  df_attack = df_attack.iloc[1: , :].astype({"time":int})
  df_attack = df_attack[:int(training_info.total_data_size*.4)]

  # Split data for base and transfer leanring
  df_normal_base, df_normal_trim = split_data(df_normal, training_info.base_transfer_split)
  df_attack_base, df_attack_trim = split_data(df_attack, training_info.base_transfer_split)

  # Combine normal data set with attack set
  df_base = df_normal_base._append(df_attack_base)

  # Reshape base model data
  (np_base_train_x, np_base_test_x, np_base_train_y, np_base_test_y) = reshape_data(df_base, training_info, training_info.base_training_info.train_test_split)

  # Combine transfer data and add trim data
  df_transfer_with_trim = add_trim_data(df_normal_trim, df_attack_trim)

  # Reshape transfer model data
  (np_transfer_train_x, np_transfer_test_x, np_transfer_train_y, np_transfer_test_y) = reshape_data(df_transfer_with_trim, training_info, training_info.transfer_training_info.train_test_split)

  # Return data
  return (np_base_train_x, np_base_test_x, np_base_train_y, np_base_test_y, np_transfer_train_x, np_transfer_test_x, np_transfer_train_y, np_transfer_test_y)

# Models

In [ ]:
def generate_base_model(training_info):
  model = Sequential()
  model.add(LSTM(256, activation='relu', kernel_regularizer=regularizers.l2(0.0001), return_sequences=True, input_shape=(training_info.seq, training_info.features)))
  model.add(BatchNormalization())
  model.add(Dropout(0.3))
  model.add(LSTM(512, activation='relu', kernel_regularizer=regularizers.l2(0.0001), return_sequences=True))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))
  model.add(LSTM(1024, activation='relu', kernel_regularizer=regularizers.l2(0.0001), return_sequences=True))
  model.add(BatchNormalization())
  model.add(Dropout(0.3))
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))

  optimizer = tf.keras.optimizers.Adam(learning_rate=training_info.base_training_info.learning_rate)
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['acc',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

  # Write model shape to logs
  with open(results_folder_path + "/base_model_shape.txt", 'w') as f:
    # Redirect the output of model.summary() to the file
    model.summary(print_fn=lambda x: f.write(x + '\n'))

  return model

def generate_transfer_model(base_model, training_info):
  # Create a new model
  transfer_model = Sequential()
  transfer_model.add(Input(shape=(training_info.seq, training_info.features)))

  # Add layers from the base model (except the last 2 layers)
  for layer in base_model.layers[:-2]:
    transfer_model.add(layer)
    transfer_model.layers[-1].trainable = False

  # Add new layers
  transfer_model.add(LSTM(256, activation='relu', kernel_regularizer=regularizers.l2(0.0001), return_sequences=True))
  transfer_model.add(BatchNormalization())
  transfer_model.add(Dropout(.3))
  transfer_model.add(LSTM(512, activation='relu', kernel_regularizer=regularizers.l2(0.0001), return_sequences=True))
  transfer_model.add(BatchNormalization())
  transfer_model.add(Dropout(0.2))
  transfer_model.add(Flatten())
  transfer_model.add(Dense(1, activation='sigmoid'))

  # Optimizer
  optimizer = tf.keras.optimizers.Adam(learning_rate=training_info.transfer_training_info.learning_rate)

  # Compile the transfer model
  transfer_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

  # Write model shape to logs
  with open(results_folder_path + "/transfer_model_shape.txt", 'w') as f:
    # Redirect the output of model.summary() to the file
    transfer_model.summary(print_fn=lambda x: f.write(x + '\n'))

  return transfer_model

# Training and Logging

In [ ]:
def create_and_log_plots(history, results_folder_path, model_type):
  # Graph Results
  print(history.history.keys())

  # Summarize history for accuracy
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.savefig(results_folder_path + "/" + model_type + "_accuracy.png")
  plt.clf()

  # Summarize history for loss
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.savefig(results_folder_path + "/" + model_type + "_loss.png")
  plt.clf()

  # Summarize history for precision
  plt.plot(history.history[[s for s in history.history.keys() if 'precision' in s][0]])
  plt.plot(history.history[[s for s in history.history.keys() if 'val_pr' in s][0]])
  plt.title('model precision')
  plt.ylabel('precision')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.savefig(results_folder_path + "/" + model_type + "_precision.png")
  plt.clf()

  # Summarize history for loss
  plt.plot(history.history[[s for s in history.history.keys() if 'recall' in s][0]])
  plt.plot(history.history[[s for s in history.history.keys() if 'val_rec' in s][0]])
  plt.title('model recall')
  plt.ylabel('recall')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.savefig(results_folder_path + "/" + model_type + "_recall.png")
  plt.clf()

def train_test_model(model, training_info, df_train_x, df_test_x, df_train_y, df_test_y, results_folder_path, model_type):
  # Add folder for model type
  results_folder_path = results_folder_path + "/" + model_type
  os.mkdir(results_folder_path)

  # Train
  epochs = training_info.base_training_info.epochs if (model_type == 'base') else training_info.transfer_training_info.epochs
  val_split = (1 -(training_info.base_training_info.train_val_split if (model_type == 'base') else training_info.transfer_training_info.train_val_split))
  batch_size = training_info.batch_size

  start = time.time()
  with open(results_folder_path + "/training_output.txt", 'w') as f:
    history = model.fit(df_train_x, df_train_y, validation_split = val_split, batch_size=batch_size, epochs=epochs,  callbacks=[LambdaCallback(on_epoch_end=lambda epoch, logs: f.write(str(logs) + '\n'))])
    f.write(f"Training time: {time.time() - start}")

  # Save model
  model.save(results_folder_path + "/base_model.keras")

  # Logg Training
  history_df = pd.DataFrame(history.history)
  history_df.to_csv(results_folder_path+'/training_history.csv', index=False)

  # Create and log plots
  create_and_log_plots(history, results_folder_path, model_type)

  # Test
  start = time.time()
  y_pred = model.predict(df_test_x, batch_size=batch_size, verbose=1)
  end = time.time()
  # Log time to file
  with open(results_folder_path + "/speed.txt", "w") as f:
    f.write(f"Time {end-start}, predictions ({len(y_pred)}), pred/s: {(len(y_pred)/(end-start))}")
  display(y_pred)
  print(len(y_pred))

  threshold = 0.5  # You can adjust this threshold
  y_pred_binary = (y_pred > threshold).astype(int)
  #y_pred_bool = np.argmax(y_pred, axis=1)
  display(y_pred_binary)
  display(np.count_nonzero(y_pred_binary == 0))
  display(np.count_nonzero(y_pred_binary == 1))
  print(len(y_pred_binary))

  #df_base_test_y_labels = np.argmax(df_test_y, axis=1)

  # Log classification report to file
  #with open(results_folder_path + "/classification_report.txt", "w") as f:
  report = classification_report(df_test_y, y_pred_binary, digits=10, output_dict=True)
  df = pd.DataFrame(report).transpose()
  df.to_csv(results_folder_path + "/classification_report.csv", index=True)

# Logic for Iterating Through Test Param CSV

In [ ]:
# Load in CSV
training_info_list = generate_training_info_list_from_csv('gdrive/My Drive/Thesis/training_info.csv')
results_path = 'gdrive/MyDrive/Thesis/Results'
print(len(training_info_list))
# Run Training
for training_info in training_info_list:
  # Create Results Folder
  results_folder_path = results_path + "/" + generate_folder_name_from_training_info(training_info);
  print(results_folder_path)
  os.mkdir(results_folder_path)

  # Log test setup info to results path\
  save_training_info_as_json(training_info, results_folder_path, 'training_info.json')

  # Grab data
  (np_base_train_x, np_base_test_x, np_base_train_y, np_base_test_y, np_transfer_train_x, np_transfer_test_x, np_transfer_train_y, np_transfer_test_y) = get_formated_and_split_data(training_info)

  # Build Base Model
  base_model = generate_base_model(training_info)
  display(np_base_train_y)
  # Train and Test Base Model and Log Results
  train_test_model(base_model, training_info, np_base_train_x, np_base_test_x, np_base_train_y, np_base_test_y, results_folder_path, "base")

  # Build Transfer Mode
  transfer_model = generate_transfer_model(base_model, training_info)

  # Train and Test Transfer Model and Log Results
  train_test_model(transfer_model, training_info, np_transfer_train_x, np_transfer_test_x, np_transfer_train_y, np_transfer_test_y, results_folder_path, "transfer")

  # # Clear data****
  # clear_data()


62
gdrive/MyDrive/Thesis/Results/Attack_rpm-steps_16-feat_10-seq_40-substeps_2-batchsize_128-classes_2-basetransplit_0.7-datasize_6000000-baseepoch_20-basettsplit_0.8-basetvsplit_0.7-transepoch_10-transttsplit_0.8-transtvsplit_0.7_241124-011651


array([1, 0, 0, ..., 0, 1, 0])

array([1, 0, 0, ..., 0, 1, 1])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


array([1, 0, 0, ..., 1, 0, 1])

...

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

1094

685

1779


<Figure size 640x480 with 0 Axes>